In [1]:
!pip install replicate

# ✅ Install dependencies
!pip install gradio langchain langchain_community faiss-cpu transformers

import os
import gradio as gr
import tempfile
from langchain.vectorstores import FAISS
from langchain_community.llms import Replicate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.embeddings import HuggingFaceEmbeddings

# 🔑 Set Replicate token securely
os.environ['REPLICATE_API_TOKEN'] = input("Enter your Replicate API token: ")

# IBM Granite model via Replicate
model_path = "ibm-granite/granite-3.3-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = Replicate(model=model_path, replicate_api_token=os.environ['REPLICATE_API_TOKEN'])

# 📚 Sample LearnMate knowledge base
filename = "learnmate_courses.txt"
with open(filename, "w") as f:
    f.write("""
Frontend Development Pathway:
1. HTML & CSS Fundamentals (Beginner) — Learn basic web page structure and styling.
2. JavaScript Essentials (Beginner) — Add interactivity to websites.
3. React for Developers (Intermediate) — Build dynamic, modern web apps.
4. Frontend Capstone Project (Advanced) — Create a real-world portfolio project.

Cybersecurity Pathway:
1. Intro to Cybersecurity (Beginner) — Understand basic security concepts.
2. Network Security (Intermediate) — Learn to secure communication channels.
3. Ethical Hacking (Advanced) — Test systems for vulnerabilities ethically.

UI/UX Design Pathway:
1. Intro to UI/UX (Beginner) — Learn the basics of user interface and experience.
2. Wireframing & Prototyping (Intermediate) — Create interactive prototypes.
3. UI/UX Capstone (Advanced) — Complete a design project for a real client.

Tips for Learning:
- Always build projects alongside learning theory.
- Review and adapt your learning roadmap every 2 months.
- Start from your current skill level and progress gradually.
""")

# 📄 Load & Split content
loader = TextLoader(filename)
documents = loader.load()
splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)
texts = splitter.split_documents(documents)

# 🔍 Vector store (FAISS for quick Colab testing)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(texts, embeddings)

# 🎯 Prompt for LearnMate
template = """
You are LearnMate, an AI learning coach.
User Question: {question}
Based on the student's interests, skill level, and goals, give a personalized learning pathway.
Suggest specific courses, explain why they fit, and provide tips for progression.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# 🔗 Chains
llm_chain = LLMChain(llm=model, prompt=prompt)
combine_chain = StuffDocumentsChain(llm_chain=llm_chain)

rag_chain = RetrievalQA(
    retriever=vector_db.as_retriever(),
    combine_documents_chain=combine_chain,
    return_source_documents=False
)

# 💬 Gradio Interface
def ask_learnmate(query):
    try:
        response = rag_chain.run(query)
        return response
    except Exception as e:
        return f"❌ Error: {str(e)}"

iface = gr.Interface(
    fn=ask_learnmate,
    inputs=gr.Textbox(label="Ask LearnMate", placeholder="e.g. How do I become a frontend developer?"),
    outputs=gr.Textbox(label="LearnMate's Advice"),
    title="LearnMate — Personalized Course Pathways",
    description="Get a personalized learning roadmap for your career goals in frontend development, cybersecurity, or UI/UX.",
    theme="default"
)

iface.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
Enter your Replicate API token: r8_Kyu1I3OCTNrkwdAY9wh0xqVfm7r9H3o0PNpri


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-2139498975.py:73: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=prompt)
/tmp/ipython-input-2139498975.py:74: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_chain = StuffDocumentsChain(llm_chain=llm_chain)
/tmp/ipython-input-2139498975.py:76: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/
  rag_chain = RetrievalQA(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a3d2800bc961581fb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
